# Homework 4 

## Joining Data

In [193]:
import pandas as pd
import numpy as np

In [194]:
cell_df = pd.read_csv('https://raw.githubusercontent.com/su-mt4007/data/refs/heads/main/cell_phones_total.csv')

We start by cleaning the data as we did in homework 3.

In [ ]:
for cell in range(len(cell_df)):
    if np.isnan(cell_df.loc[cell,'1960']):
        cell_df.loc[cell,'1960'] = 0.0

cleaned_cell_df = cell_df.fillna(method='ffill', axis= 1)

cleaned_cell_df.iloc[:,1:] = cleaned_cell_df.iloc[:,1:]\
    .replace({'k' :'e3', 'M' : 'e6', 'B': 'e9'}, regex = True)\
    .astype(float)

In [ ]:
pop_df = pd.read_csv('https://raw.githubusercontent.com/su-mt4007/data/refs/heads/main/pop_data.csv')

We want to know if there is a row that don't have any information about the population at all.

In [ ]:
rows_with_all_nan = pop_df.iloc[:, 2:].isna().all(axis=1)

for i in range(len(rows_with_all_nan)):
    if rows_with_all_nan[i]:
        print('The row', i, 'only contains Nan')

We can see from the output that row 110 only contains Nan. Because of this we choose to delete that row from the data frame. The first column is also unnecessary so we remove that column aswell.

In [ ]:
cleaned_pop_df = pop_df.drop(index=110)\
    .reset_index(drop = True)\
    .iloc[:, 1:]

In [ ]:
cleaned_pop_df

If a country is missing data in 1960 we don't want to set the value to 0 because that seems improbible, later we will also devide the amount of cell phones with the population and we can't devide by 0. Because of this we choose to backward fill. First we want to know if there is some minning values in the year 2020.

In [ ]:
nan_in_2020 = cleaned_pop_df.iloc[:, 61].isna()
for i in range(len(nan_in_2020)):
    if nan_in_1960[i]:
        print('The row', i, 'has Nan in the column 2020')

Because we don't get an output we know that all the countries have data in the year 2020 so we can now backward fill.

In [ ]:
clean_pop_df = cleaned_pop_df.bfill(axis=1)

Now we want to pivot the data frame.

In [ ]:
pivot_pop_df = clean_pop_df.melt(id_vars ='iso-3', var_name='year', value_name='population')
pivot_pop_df.head()

We now do the same for the cell phones.

In [ ]:
pivot_cell_df = cleaned_cell_df.melt(id_vars='iso-3', var_name='year', value_name='n_cellphones')
pivot_cell_df.head()

Now, we join the two data frames and presents the table asked for in exercise.

In [ ]:
cell_pop_df = pivot_cell_df.merge(pivot_pop_df, on =['iso-3','year'], how ='inner')

cell_pop_df.head()

We read the country data.

In [ ]:
country_df = pd.read_csv('https://raw.githubusercontent.com/su-mt4007/data/refs/heads/main/country_data.csv')\
    .rename(columns={'alpha-3':'iso-3'})

country_df.head()

We join the country data with the cell phone and population data.

In [ ]:
merged_df = country_df.merge(cell_pop_df, on='iso-3')
merged_df.head()

We creat a new column for phones per capita.

In [ ]:
merged_df['phones per capita']= merged_df['n_cellphones']/merged_df['population']

In [ ]:
pivot_df = merged_df.pivot(columns=['year', 'name'])
pivot_df.head()

Creat the table asked for.

In [ ]:
merged_df[['name', 'year','phones per capita']].sort_values('phones per capita').head() 